# PydanticOutputParser
- `get_format_instructions()` : 어떤형식으로 출력할지 지침을 알려준다. ex) JSON...
- `parse()` : Parse를 가지고 구조화된 객체로 변환시켜준다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain_teddynote.messages import stream_response

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [5]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [6]:
llm = ChatOpenAI(model='gpt-4o-mini')
prompt = PromptTemplate.from_template("다음의 이메일 내용중 중요한 이메일을 추출해 주세요. \n\n{email_conversation}")
chain = prompt | llm


In [7]:
answer = chain.stream({"email_conversation": email_conversation})
final_answer = ''
for token in answer:
    print(token.content, end="", flush=True )
    final_answer += token.content

중요한 이메일 내용은 다음과 같습니다:

- **보낸 사람:** 김철수 (chulsoo.kim@bikecorporation.me)
- **받는 사람:** 이은채 (eunchae@teddyinternational.me)
- **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- **요청사항:** ZENESIS 모델에 대한 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보)
- **미팅 제안:** 다음 주 화요일(1월 15일) 오전 10시에 귀사 사무실에서 미팅 제안

이 내용은 자전거 유통 협력과 관련된 중요한 요청 및 미팅 일정 제안이 포함되어 있습니다.

In [8]:
print(final_answer)

중요한 이메일 내용은 다음과 같습니다:

- **보낸 사람:** 김철수 (chulsoo.kim@bikecorporation.me)
- **받는 사람:** 이은채 (eunchae@teddyinternational.me)
- **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- **요청사항:** ZENESIS 모델에 대한 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보)
- **미팅 제안:** 다음 주 화요일(1월 15일) 오전 10시에 귀사 사무실에서 미팅 제안

이 내용은 자전거 유통 협력과 관련된 중요한 요청 및 미팅 일정 제안이 포함되어 있습니다.


### PydanticOutputParser 선언

In [9]:
# from langchain_core.pydantic_v1 import BaseModel, Field
# BaseModel을 상속받는다. 변수에다가 Field를 대입하고, description에다가 원하는 정보에 대한 설명구를 자세하게 적어줘야 한다.
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
#from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [10]:
# get_format_instructions() : 언어모델이 출력해야 할 정보의 형식을 알려준다.
# parse(): 문자열을 넣어주면, 형식으로 변환해준다.

In [11]:
# get_format_instructions() : schema
# 우리는 이렇게 작성할 수 없다. 따라서 class EmailSummary(BaseModel)처럼 적으면 알아서 바꿔준다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "type": "string"}, "date": {"title": "Date", "description"

## 프롬프트 정의
1. `question`: 유저의 질문, 요청사항을 받는다.
2. `email_conversation` : 이메일 본문의 내용을 입력
3. `format` : 형식을 지정. `get_format_instructions`이 들어간다

In [12]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the folloowing questions in KOREAN.

#QUESTION:
{question}

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""

)
# format에 PydanticOutputParser의 부분 포맷팅(partial)추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [13]:
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"title": "Person", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c\\uc758 \\uc774\\uba54\\uc77c \\uc8fc\\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\\uba54\\uc77c \\uc81c\\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\\

In [14]:
chain = prompt | llm

In [35]:
print(
    chain.invoke({
    "email_conversation": email_conversation,
    "question": "이메일 내용중 주요만 추출해 주세요"
})
)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='바이크코퍼레이션의 김철수가 ZENESIS 자전거의 기술 사양, 배터리 성능, 디자인 정보를 요청하며, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안함.' date='2024-01-15'


In [15]:
response = chain.stream(
    {
        "email_conversation" : email_conversation,
        "question" : "이메일 내용중 주요 내용을 추출해 주세요"
    }
)
response

<generator object RunnableSequence.stream at 0x0000020407F8DF30>

In [16]:
final_response = ''
for token in response:
    final_response += token.content
    print(token.content, end="", flush=True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 이은채 대리님에게 ZENESIS 자전거에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.",
  "date": "2024-01-15"
}
```

In [17]:
type(final_response)

str

In [18]:
structured = parser.parse(final_response)

In [19]:
print(structured.person)
print(structured.email)

김철수
chulsoo.kim@bikecorporation.me


In [20]:
chain = prompt | llm | parser

In [21]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요."
    }
)
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리에게 'ZENESIS' 자전거의 브로슈어 요청과 함께 협력 가능성을 논의하기 위한 미팅을 제안함.", date='2024-01-15T10:00:00')

In [22]:
response.email

'chulsoo.kim@bikecorporation.me'

In [23]:
response.person

'김철수'

## LLM에 구조화 된 출력 추가
`.with_structured_output(Pydantic)`을 사용하여 출력파서를 추가하면, 출력 Pydantic객체로 변환할 수 있다. 그러나 stream출력은 되지 않는다.

In [24]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
#from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [25]:
llm_with_structered = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini'
).with_structured_output(EmailSummary)

In [26]:
print(email_conversation)

From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션



In [27]:
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 유통 협력 제안을 하며, 'ZENESIS' 모델에 대한 브로슈어 요청과 미팅 일정을 제안하는 내용입니다.", date='2024-01-15T10:00:00')

In [28]:
print(answer.person)
print(answer.email)

김철수
chulsoo.kim@bikecorporation.me


In [29]:
llm_with_structered.invoke(email_conversation)

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 유통 협력 제안을 하며, 'ZENESIS' 모델에 대한 브로슈어 요청과 미팅 일정을 제안하는 내용입니다.", date='2024-01-15T10:00:00')